In [2]:
#%pip install -r requirements.txt

import pandas as pd
import numpy as np

In [8]:
#interest rate

df = pd.read_csv('./data/before-convert/interest_rate.csv')
df['year'] = pd.DatetimeIndex(df['TIME']).year
df['month'] = pd.DatetimeIndex(df['TIME']).month

df = df.drop('TIME', axis=1)

df = df.rename(columns = {'LOCATION':'location'})
df = df.drop('Country', axis=1)

df.loc[df['location'] == 'GBR' , 'location'] = 'EU'

longTerm = df.query('Subject == "Long-term interest rates, Per cent per annum"')
longTerm = longTerm.rename(columns = {'Interest-Rate':'interest_rate_long_term'})
longTerm = longTerm.drop('Subject', axis=1)

shortTerm = df.query('Subject == "Short-term interest rates, Per cent per annum"')
shortTerm = shortTerm.rename(columns = {'Interest-Rate':'interest_rate_short_term'})
shortTerm = shortTerm.drop('Subject', axis=1)


immediateTerm = df.query('Subject == "Immediate interest rates, Call Money, Interbank Rate, Per cent per annum"')
immediateTerm = immediateTerm.rename(columns = {'Interest-Rate':'interest_rate_immediate_term'})
immediateTerm = immediateTerm.drop('Subject', axis=1)

df = pd.merge(longTerm, shortTerm, on=['location', 'year', 'month'] , how='inner', suffixes=('', '_delme'))
df = pd.merge(df, immediateTerm, on=['location', 'year', 'month'] , how='inner', suffixes=('', '_delme'))
df = df[[c for c in df.columns if not c.endswith('_delme')]]

df.to_csv('./data/interest_rate.csv', index=False)

In [13]:
#cpi

#au
df_au = pd.read_csv('./data/before-convert/cpi_au.csv')

df_au['Date'] = pd.DatetimeIndex(df_au['TIME'])
df_au = df_au.drop('TIME', axis=1)

df_au['Date'] = pd.to_datetime(df_au['Date']).dt.to_period('M')
df_au = df_au.set_index('Date').resample('M').interpolate()
df_au['location'] = 'AUS'
df_au['year'] = pd.DatetimeIndex(df_au.index.to_timestamp()).year
df_au['month'] = pd.DatetimeIndex(df_au.index.to_timestamp()).month

df_au.reset_index(drop=True, inplace=True)


#us
df_us = pd.read_csv('./data/before-convert/cpi_us.csv')

df_final_us = pd.DataFrame(columns=['year','month'])

for index, row in df_us.iterrows():
    for i in range(12):
        df_final_us = pd.concat([df_final_us, pd.DataFrame.from_records([{'year':row[0],'month':i+1, 'cpi-index':row[i+1]}])])

df_final_us['location'] = 'USA'
df_final_us = df_final_us.dropna()
df_final_us['year'] = df_final_us['year'].astype(int)


#eu

df_eu = pd.read_csv('./data/before-convert/cpi_eu.csv')

df_eu['year'] = pd.DatetimeIndex(df_eu['Date'], dayfirst=True).year
df_eu['month'] = pd.DatetimeIndex(df_eu['Date'], dayfirst=True).month
df_eu = df_eu.drop('Date', axis=1)

df = pd.concat([df_au, df_final_us,df_eu])


df.to_csv('./data/cpi.csv', index=False)

In [21]:
#energy_price
lagged_months = 48

#au

df_au = pd.read_csv('./data/before-convert/energy_price_au.csv')

df_au['year'] = pd.DatetimeIndex(df_au['Date'], dayfirst=True).year
df_au['month'] = pd.DatetimeIndex(df_au['Date'], dayfirst=True).month

df_au = df_au.rename(columns = {'Value':'energy_price'})


df_au['Date'] = pd.to_datetime(df_au['Date'],dayfirst=True)
df_au = df_au.set_index('Date').resample('M').median(numeric_only=True)

df_au.head(100)

for i in range(lagged_months):
    print('energy_price_lag_%sm'%(i+1))
    df_au['energy_price_lag_%sm'%(i+1)] = df_au['energy_price'].shift(freq= '%sM'%(i+1))

df_au.reset_index(drop=True, inplace=True)

df_au['year'] = df_au['year'].astype(int)
df_au['month'] = df_au['month'].astype(int)
df_au['location'] = 'AUS'

#us

df_us = pd.read_csv('./data/before-convert/energy_price_us.csv')

df_us['year'] = pd.DatetimeIndex(df_us['date'], dayfirst=True).year
df_us['month'] = pd.DatetimeIndex(df_us['date'], dayfirst=True).month
df_us['location'] = 'USA'
df_us = df_us.rename(columns = {'value':'energy_price'})
df_us = df_us.set_index('date')

for i in range(lagged_months):
    df_us['energy_price_lag_%sm'%(i+1)] = df_us['energy_price'].shift(i+1)

df_us.reset_index(drop=True, inplace=True)


#eu

df_eu = pd.read_csv('./data/before-convert/energy_price_eu.csv')
df_eu['year'] = pd.DatetimeIndex(df_eu['Date'], dayfirst=True).year
df_eu['month'] = pd.DatetimeIndex(df_eu['Date'], dayfirst=True).month
df_eu['location'] = 'EU'
df_eu = df_eu.rename(columns = {'Median':'energy_price'})

df_eu = df_eu.drop('Date', axis=1)
df_eu = df_eu.drop('Open', axis=1)
df_eu = df_eu.drop('High', axis=1)
df_eu = df_eu.drop('Low', axis=1)
df_eu = df_eu.drop('Close', axis=1)

for i in range(lagged_months):
    df_eu['energy_price_lag_%sm'%(i+1)] = df_eu['energy_price'].shift(i+1)

df = pd.concat([df_au, df_us, df_eu])

print(df)

df = df.dropna()

df.to_csv('./data/energy_price.csv', index=False)

energy_price_lag_1m
energy_price_lag_2m
energy_price_lag_3m
energy_price_lag_4m
energy_price_lag_5m
energy_price_lag_6m
energy_price_lag_7m
energy_price_lag_8m
energy_price_lag_9m
energy_price_lag_10m
energy_price_lag_11m
energy_price_lag_12m
energy_price_lag_13m
energy_price_lag_14m
energy_price_lag_15m
energy_price_lag_16m
energy_price_lag_17m
energy_price_lag_18m
energy_price_lag_19m
energy_price_lag_20m
energy_price_lag_21m
energy_price_lag_22m
energy_price_lag_23m
energy_price_lag_24m
energy_price_lag_25m
energy_price_lag_26m
energy_price_lag_27m
energy_price_lag_28m
energy_price_lag_29m
energy_price_lag_30m
energy_price_lag_31m
energy_price_lag_32m
energy_price_lag_33m
energy_price_lag_34m
energy_price_lag_35m
energy_price_lag_36m
energy_price_lag_37m
energy_price_lag_38m
energy_price_lag_39m
energy_price_lag_40m
energy_price_lag_41m
energy_price_lag_42m
energy_price_lag_43m
energy_price_lag_44m
energy_price_lag_45m
energy_price_lag_46m
energy_price_lag_47m
energy_price_lag_48m
 